# Inference


In [ ]:
!mkdir dataset


In [ ]:
!unzip "/content/drive/My Drive/Pulse Classification/Indian Pulses.zip" -d "/content/dataset"

In [ ]:
!pip install -q albumentations==0.4.5
!pip install -q efficientnet_pytorch

     |████████████████████████████████| 122kB 5.5MB/s 
     |████████████████████████████████| 634kB 47.5MB/s 


In [9]:
import os
import cv2

import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from efficientnet_pytorch import EfficientNet
import albumentations as A
from torch.utils.data.sampler import SequentialSampler
from albumentations.pytorch import ToTensorV2

## Model

In [10]:
def Net(model_name = 'b0', output = 14):
    model = EfficientNet.from_pretrained(f'efficientnet-{model_name}')
    model._fc = nn.Linear(in_features = model._fc.in_features, out_features = output, bias = True)
    return model

In [ ]:
def load_model(path):
  model = Net().cuda()
  model.load_state_dict(torch.load(path)["model_state_dict"])
  model.eval()
  return model


## Dataloader

In [38]:
class DatasetRetriever(Dataset):

    def __init__(self, image_ids, transforms=None):
        super().__init__()
        self.image_ids = image_ids

        self.transforms = transforms

    def __getitem__(self, idx: int):
        image_id = self.image_ids[idx]
        image = cv2.imread(f'{Config.DIR}/{image_id}', cv2.IMREAD_COLOR)

        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']

        return image

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [36]:
def validation_augmentations(img_size = 224):
    return A.Compose([
            A.Resize(height=img_size, width=img_size, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

## Configuration

In [39]:
class Config:

  DIR = "/content/dataset"

  num_workers=4
  batch_size=32

## Engine

In [91]:
class Engine:

  def __init__(self, fold):
    
    self.predictions = list()
    self.model = load_model(f'/content/224_b0_{fold}.pt')
    print(f'Model loaded for fold {fold}')
  
  def fit(self, validation_loader):
    for _, x_val in enumerate(validation_loader):
      
      temp = list()
      images = x_val.to(torch.device('cuda'), dtype=torch.float32)
      
      pred = self.model(images)
      y_pred = nn.functional.softmax(pred, dim=1).data.cpu().numpy()

      for each_pred in y_pred:
        temp.append(each_pred.argmax())

      self.predictions.extend(temp)
    return self.predictions


## Trainer

In [102]:
def perform_for_fold(fold_number=0):

  valid_X = df[df["k-fold"] == fold_number].Path.values
  valid_Y = df[df["k-fold"] == fold_number].Label_enc.values

  valid_dataset = DatasetRetriever(valid_X, validation_augmentations(img_size = 224))
        
  validation_loader = torch.utils.data.DataLoader(
            valid_dataset, 
            batch_size=Config.batch_size,
            num_workers=Config.num_workers,
            shuffle=False,
            sampler=SequentialSampler(valid_dataset),
            pin_memory=False,
        ) 
  
  engine = Engine(fold_number)
  pred = engine.fit(validation_loader)
  return pred, valid_Y

## Inference started for all folds

In [ ]:
df = pd.read_csv('/content/dataset_folds.csv')

In [103]:
pred_0, true_0=perform_for_fold(0)
pred_1, true_1=perform_for_fold(1)
pred_2, true_2=perform_for_fold(2)
pred_3, true_3=perform_for_fold(3)
pred_4, true_4=perform_for_fold(4)

Loaded pretrained weights for efficientnet-b0
Model loaded for fold 0
Loaded pretrained weights for efficientnet-b0
Model loaded for fold 1
Loaded pretrained weights for efficientnet-b0
Model loaded for fold 2
Loaded pretrained weights for efficientnet-b0
Model loaded for fold 3
Loaded pretrained weights for efficientnet-b0
Model loaded for fold 4


## Evaulation

In [116]:
pred_all = np.concatenate((pred_0, pred_1, pred_2, pred_3, pred_4), axis=0)
true_all = np.concatenate((true_0, true_1, true_2, true_3, true_4), axis=0)

In [118]:
import json

def key_conversion(array):
  return [INV_KEY[each]for each in array]

with open('/content/label_key.json') as f:
  KEY = json.load(f)

INV_KEY = {v: k for k, v in KEY.items()}

pred_label = key_conversion(pred_all)
true_label = key_conversion(true_all)

In [125]:
from sklearn.metrics import classification_report

print(classification_report(true_label, pred_label))

                            precision    recall  f1-score   support

              Adzuki Beans       0.99      0.98      0.98       632
                Black Gram       0.96      0.97      0.97       527
                 Chickpeas       0.99      0.99      0.99       622
                  Dew Bean       0.98      0.97      0.98       398
           Green Chickpeas       0.98      0.98      0.98       551
                Green Gram       0.98      0.97      0.98       564
               Pinto Beans       0.99      0.99      0.99       765
          Red Kidney Beans       0.99      0.99      0.99       577
               Red Lentils       0.99      0.99      0.99       586
Split & Skinned Black Gram       0.99      0.99      0.99       557
          Split Black Gram       0.96      0.95      0.96       488
          Split Green Gram       0.97      0.97      0.97       220
        White Kidney Beans       0.99      0.99      0.99       575
            Yellow Lentils       0.98      0.99